# N starvation 

## Which N sources allow MED4 to grow

check out Escher visualization https://escher.readthedocs.io/en/latest/getting_started.html

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import seaborn as sns
import itertools
from matplotlib.colors import LogNorm, Normalize


In [2]:
import subprocess
import time


In [3]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import production_envelope


In [4]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [5]:
sns.hls_palette

<function seaborn.palettes.hls_palette(n_colors=6, h=0.01, l=0.6, s=0.65, as_cmap=False)>

In [6]:
# values from Ofaim at el.

PARAMETER_VALUES = [#"Name",     "Reaction ID",          "Lower bound", "UpperBound"
                   ["HCO3",      "HCO3EXcar",            -8,            0],
                   #["Nitrogen",  "AmmoniaEX",            -0.56,         0],
                   #["Phosphate", "FAKEOrthophosphateEX", -0.1,          0],
                   ["Light",     "LightEX",              -150,          0]]
CO2MAX = 0.82

# Block fake reactions
FAKE_TRANSPORT = ["AminosugarsOUT", "FAKEAAOUT", "FAKEABPOUT", "FAKEacpTRANS", "FAKEApoacpTRANS", "FAKEThioredoxinTRANS", 
                  "FreefattyacidsOUT", "7NMeth7carbOUT", "ArtificialproteinOUT", "FADOUT", "LipoylproteinTRANS", "MenaquinoneOUT", 
                  "NicotinateOUT", "THFpolyglutOUT", "Thiamin_dpOUT"]


In [68]:
def get_elements(rid):
    r = model.reactions.get_by_id(rid)
    m = r.reactants[0]    
    elements = m.elements.copy()
    
    res = {
        'reaction' : rid,
        'metabolite' : m.name,
        'N' : 'N' in elements,
        'C' : 'C' in elements,
        
    }
    return pd.Series(res)

# Import model and manipulate based on Ofaim at el

In [8]:
import os
model_dpath = os.path.join('..', 'Model_files')
model_fname = 'iSO595v7.xml'
model_fpath = os.path.join(model_dpath, model_fname)


In [9]:
model = cobra.io.read_sbml_model(model_fpath)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [10]:
# manipulations copied from Ofaim at el.

remove_blocked = False

# Block H2S
model.reactions.H2SEX.lower_bound = 0

# Block fake transports
for rid in FAKE_TRANSPORT:
    model.reactions.get_by_id(rid).bounds = (0,0)

# Remove blocked reactions
if remove_blocked:
    blocked = cobra.flux_analysis.find_blocked_reactions(model, open_exchanges = True)
    print('blocked', len(blocked), blocked)
    model.remove_reactions([model.reactions.get_by_id(r_id) for r_id in blocked])

# Block maximum CO2 production
model.reactions.CO2EX.bounds = (0, CO2MAX)

for i, row in enumerate(PARAMETER_VALUES):
    # Row: Name, Reaction ID, lower bound, upper bound
    key = row[0]
    reaction_id = row[1]
    lower_bound = row[2]
    upper_bound = row[3]
    r = model.reactions.get_by_id(reaction_id)
    # Fix flux
    r.bounds = (lower_bound, upper_bound)

In [11]:
exchange_ids = [r.id for r in model.exchanges] # + ["R00024"]

In [12]:
print('reactions', len(model.reactions))
print('metabolites', len(model.metabolites))
print('genes', len(model.genes))

reactions 994
metabolites 802
genes 595


In [13]:
model

Name,COBRAModel
Memory address,0x07fc706b40940
Number of metabolites,802
Number of reactions,994
Number of groups,55
Objective expression,1.0*BIOMASS - 1.0*BIOMASS_reverse_69053
Compartments,"Cytoplasm, Extracellular, unknownCompartment2, Periplasm, unknownCompartment5"


In [14]:
model.metabolites.get_by_id('Carbamate[c]')

Metabolite identifier,Carbamate[c]
Name,Carbamate
Memory address,0x7fc706b017c0
Formula,CH3NO2
Compartment,c
In 4 reaction(s),"R01395, R03546, R07316, R10949"


In [15]:
cobra.Configuration()

Attribute,Description,Value
solver,Mathematical optimization solver,glpk
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,11
cache_directory,Path for the model cache,/home/oweissberg/.cache/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


In [19]:
model.reactions.get_by_id('BIOMASS')

Reaction identifier,BIOMASS
Name,Biomass formation
Memory address,0x7f1e197d0e50
Stoichiometry,104.0 ATP[c] + 0.0324116276714271 BioPool[c] + 0.0268408791654006 Bmineral[c] + 0.0559910868023904 Cell_wall[c] + 0.00111921401802897 Free_nucleic_acids[c] + 104.0 H2O[c] + 0.649245416793275... 104.0 ATP + 0.0324116276714271 BioPool + 0.0268408791654006 Trace metals and minerals + 0.0559910868023904 Cell wall Biomass precursor + 0.00111921401802897 Free nucleic acids + 104.0 H2O +...
GPR,
Lower bound,0.0
Upper bound,1000.0


# Compare exchange reactions/metabolites of the two models

In [69]:
exchange_ids = [r.id for r in model.exchanges] 
pro_med_df = pd.DataFrame({'mediumExchange' : exchange_ids})
pro_med_df = pro_med_df.mediumExchange.apply(get_elements)


In [72]:
#pro_df = pd.read_excel('PRO_N_sources.xlsx', sheet_name='secretion')
alt_dpath = '../../HOT1A3-model/N_Starvation/'
alt_med_df = pd.read_excel(os.path.join(alt_dpath, 'HOT1A3_medium.xlsx'))


In [70]:
pro_med_df.head()

,reaction,metabolite,N,C
0,AminosugarsEX,Amino sugars pool,False,False
1,FAKEAAEX,amino_acid_pool_e,False,False
2,FAKEABPEX,Adenosine 3_5-bisphosphate,True,True
3,FAKEacpEX,Acyl-carrier protein,False,False
4,FAKEApoacpEX,Apo-(acyl-carrier-protein),False,False


In [73]:
alt_med_df.head()

,reaction,metabolite,N,C,pro99,MED4 metabolite,promm
0,EX_actn__R_e,R Acetoin C4H8O2,False,True,NaN,NaN,NaN
1,EX_bhb_e,(R)-3-Hydroxybutanoate,False,True,NaN,NaN,NaN
2,EX_12ppd__R_e,"(R)-Propane-1,2-diol",False,True,NaN,NaN,NaN
3,EX_12ppd__S_e,"(S)-Propane-1,2-diol",False,True,NaN,NaN,NaN
4,EX_phedca_e,10 Phenyldecanoic acid,False,True,NaN,NaN,NaN


In [120]:
alt_med_df.loc[(~alt_med_df.pro99.isna()) & 
               (alt_med_df.metabolite != alt_med_df['MED4 metabolite']), #&
               #(alt_med_df.metabolite_common != alt_med_df['MED4 metabolite']),
               ['metabolite','MED4 metabolite']
               
              ].values

array([['Calcium', 'Ca2+'],
       ['Chloride', 'Cl-'],
       ['CO2 CO2', 'HCO3-'],
       ['Co2+', 'Cobalt ion'],
       ['Fe2+ mitochondria', 'Fe2+'],
       ['H2O H2O', 'H2O'],
       ['Potassium', 'K+'],
       ['Magnesium', 'Magnesium cation'],
       ['Molybdate', 'Molybdenum'],
       ['Ammonium', 'Ammonia'],
       ['O2 O2', 'O2'],
       ['Manganese', nan],
       ['Phosphate', 'Orthophosphate'],
       ['Zinc', 'Zn2+']], dtype=object)

In [122]:
pro_med_df['metabolite_common'] = pro_med_df['metabolite']
alt_med_df['metabolite_common'] = alt_med_df['metabolite']

In [123]:
pro_med_df['metabolite_common'] = pro_med_df['metabolite_common'].str.replace('[e]','', regex=False)
alt_med_df['metabolite_common'] = alt_med_df['metabolite_common'].str.replace('[e]','', regex=False)
pro_med_df['metabolite_common'] = pro_med_df['metabolite_common'].str.replace('Urea CH4N2O','Urea', regex=False)
alt_med_df['metabolite_common'] = alt_med_df['metabolite_common'].str.replace('Urea CH4N2O','Urea', regex=False)


In [124]:
mapping_exchanges = [
    ('Ca2+','Calcium'),
    ('Fe2+ mitochondria','Fe2+'),
    ('CO','Carbon monoxide'),
    ('CO2 CO2','CO2'),
    ('Cl-','Chloride'),
    ('Co2+','Cobalt ion'),
    #['CO2 CO2', 'HCO3-'],
    ('H2O H2O', 'H2O'),
    ('K+', 'Potassium',) ,
    ('Magnesium', 'Magnesium cation'),
    ('Molybdate', 'Molybdate/Molybdenum'),
    ('Molybdenum', 'Molybdate/Molybdenum'),
    ('Ammonium', 'Ammonium/Ammonia'),
    ('Ammonia', 'Ammonium/Ammonia'),
    ('O2 O2', 'O2'),
    ('Phosphate', 'Orthophosphate'),
    ('Zinc', 'Zn2+'),
    ('S_Malate', 'S_Malate/L-Malate'),
    ('L-Malate', 'S_Malate/L-Malate'),


]

for i, j in mapping_exchanges:
    pro_med_df.loc[pro_med_df['metabolite_common'].isin([i]), 'metabolite_common'] = j
    alt_med_df.loc[alt_med_df['metabolite_common'].isin([i]), 'metabolite_common'] = j
    

In [125]:
exchange_df = pd.merge(pro_med_df, alt_med_df, on='metabolite_common', how='outer', suffixes=['_pro', '_alt'])

In [126]:
exchange_df['map_status'] = 'mapped'
exchange_df.loc[exchange_df.metabolite_alt.isna(), 'map_status'] = 'PRO only'
exchange_df.loc[exchange_df.metabolite_pro.isna(), 'map_status'] = 'ALT only'
exchange_df.map_status.value_counts()

ALT only    192
mapped       59
PRO only     42
Name: map_status, dtype: int64

In [127]:
exchange_df.isna().sum()

reaction_pro         192
metabolite_pro       192
N_pro                192
C_pro                192
metabolite_common      0
reaction_alt          42
metabolite_alt        42
N_alt                 42
C_alt                 42
pro99                274
MED4 metabolite      275
promm                284
map_status             0
dtype: int64

In [128]:
exchange_df[['metabolite_common','metabolite_pro', 'metabolite_alt', 'map_status', 'reaction_pro', 'reaction_alt']].to_excel('common_exchange.xlsx')

# Look for potential cross feeding

In [163]:
pd.read_excel('PRO_N_sources.xlsx', sheet_name='uptake')


,Unnamed: 0,uptake,objective_value,reaction,metabolite,N,C
0,0,GuanosineEX,1.239447,GuanosineEX,Guanosine[e],False,False
1,1,L_AspartateEX,0.238582,L_AspartateEX,L-Aspartate,True,True
2,2,GlutathioneEX,0.137239,GlutathioneEX,Glutathione[e],False,False
3,3,L_GlutamineEX,0.121923,L_GlutamineEX,L-Glutamine,True,True
4,4,L_SerineEX,0.118972,L_SerineEX,L-Serine,True,True
5,5,L_CystineEX,0.116452,L_CystineEX,L-Cystine,True,True
6,6,L_AlanineEX,0.116452,L_AlanineEX,L-Alanine,True,True
7,7,L_LeucineEX,0.103996,L_LeucineEX,L-Leucine,True,True
8,8,GlycineEX,0.101626,GlycineEX,Glycine,True,True
9,9,UreaEX,0.098462,UreaEX,Urea,True,True


In [135]:
pro_df = pd.read_excel('PRO_N_sources.xlsx', sheet_name='secretion')
alt_dpath = '../../HOT1A3-model/N_Starvation/'
alt_df = pd.read_excel(os.path.join(alt_dpath, 'ALT_N_sources.xlsx'), sheet_name='secretion')


In [136]:
alt_df.head()

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,N_uptake,C_uptake,metabolite_secretion,N_secretion,C_secretion
0,0,sink_2ohph_c,0.0,74.900799,EX_g3ps_e,Glycerophosphoserine,True,True,2-Octaprenyl-6-hydroxyphenol,False,True
1,1,sink_4crsol_c,0.0,261.202802,EX_g3ps_e,Glycerophosphoserine,True,True,P-Cresol,False,True
2,2,sink_4hba_c,0.0,465.914339,EX_g3ps_e,Glycerophosphoserine,True,True,4-Hydroxy-benzyl alcohol,False,True
3,4,sink_hemeO_c,0.0,69.879628,EX_g3ps_e,Glycerophosphoserine,True,True,Heme O C49H56FeN4O5,True,True
4,5,sink_mobd_c,0.0,1000.000000,EX_g3ps_e,Glycerophosphoserine,True,True,Molybdate,False,False


In [153]:
p_map = pro_med_df['metabolite_common']
p_map.index = pro_med_df['metabolite']
pro_df['metabolite1_uptake'] = pro_df.metabolite_uptake.replace(p_map)
pro_df['metabolite1_secretion'] = pro_df.metabolite_secretion.replace(p_map)


In [154]:
a_map = alt_med_df['metabolite_common']
a_map.index = alt_med_df['metabolite']
alt_df['metabolite1_uptake'] = alt_df.metabolite_uptake.replace(a_map)
alt_df['metabolite1_secretion'] = alt_df.metabolite_secretion.replace(a_map)


In [155]:
alt_uptake = set(alt_df.metabolite1_uptake.unique())
pro_uptake = set(pro_df.metabolite1_uptake.unique())
alt_secretion = set(alt_df.metabolite1_secretion.unique())
pro_secretion = set(pro_df.metabolite1_secretion.unique())


In [161]:
', '.join(alt_uptake & pro_secretion)

'Putrescine, L-Valine, L-Lysine, Guanine, L-Asparagine, Glycine, L-Histidine, L-Phenylalanine, L-Glutamate, L-Alanine, L-Aspartate, L-Serine, L-Glutamine, Xanthosine, Guanosine, L-Proline, Adenine, L-Threonine, L-Isoleucine, L-Tyrosine, L-Arginine, L-Leucine'

In [162]:
', '.join(pro_uptake & alt_secretion)

'L-Valine, Urea, Ammonium/Ammonia, Guanine'

In [158]:
alt_uptake | pro_uptake | alt_secretion | pro_secretion

{'(R)-Propane-1,2-diol',
 '(S)-Propane-1,2-diol',
 '2-Aminoethylphosphonate',
 '2-Octaprenyl-6-hydroxyphenol',
 '4-Aminobutanoate',
 '4-Hydroxy-benzyl alcohol',
 '4-Hydroxybenzaldehyde',
 '4-Hydroxybenzoate',
 '4_Aminobenzoate',
 '4_Methyl_2_oxopentanoate',
 "5'-deoxyribose",
 '5_Methylthioadenosine',
 'Acetaldehyde',
 'Acetate',
 'Adenine',
 'Adenosine',
 'Ala L asp L C7H11N2O5',
 'Ala L glu L C8H13N2O5',
 'Ala-Gln',
 'Ala-His',
 'Ala-L-Thr-L',
 'Ala-Leu',
 'Allantoin',
 'Ammonium/Ammonia',
 'Benzoate',
 'Biotin',
 'Cellobiose',
 'Choline C5H14NO',
 'Citrate',
 'Cobamide coenzyme',
 'Cyanate',
 'Cys Gly C5H10N2O3S',
 'Cytidine',
 'Cytosine',
 'D-Alanine',
 'D-Alanyl-D-alanine',
 'D-Glucose',
 'D-Glyceraldehyde',
 'D-Ribose',
 'D-Serine',
 'Dethiobiotin',
 'Ethanol',
 'Ethanolamine',
 'Formaldehyde',
 'Formate',
 'Fumarate',
 'Glutathione',
 'Gly asn L C6H11N3O4',
 'Gly asp L C6H9N2O5',
 'Gly glu L C7H11N2O5',
 'Gly pro L C7H12N2O3',
 'Gly-Cys',
 'Gly-Gln',
 'Gly-Leu',
 'Gly-Phe',
 'Gl

In [159]:
pro_df.loc[pro_df.isna().sum(axis=1) > 0]

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,N_uptake,C_uptake,metabolite_secretion,N_secretion,C_secretion,metabolite1_uptake,metabolite1_secretion


In [160]:
alt_df.loc[alt_df.isna().sum(axis=1) > 0]

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,N_uptake,C_uptake,metabolite_secretion,N_secretion,C_secretion,metabolite1_uptake,metabolite1_secretion
